In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import numpy as np
import random
import numpy as np
import tensorflow as tf
import cv2
import os
import pandas as pd
from random import shuffle
from tqdm import tqdm
img_size_flat=16384
n_t_xy=np.load('/content/gdrive/My Drive/Colab Notebooks/flower_recognition/AIDD/aidd_flower.npy',allow_pickle=True)
s_t_xy=np.load('/content/gdrive/My Drive/Colab Notebooks/flower_recognition/AIDD/aidd_flower.npy',allow_pickle=True)
print(n_t_xy.shape)
n_xy=np.load('/content/gdrive/My Drive/Colab Notebooks/flower_recognition/AIDD/aidd_flower.npy',allow_pickle=True)
print(n_xy.shape)


(980, 2)
(980, 2)


'\nclass_test=s_t_xy[10:30]\nc_test_x = np.array([i[0] for i in class_test])\nc_test_x=c_test_x.reshape(-1,img_size_flat)\nc_test_y = np.array([i[1] for i in class_test])\n\nclass_test1=s_t_xy[260:280]\nc_test_x1 = np.array([i[0] for i in class_test1])\nc_test_x1=c_test_x1.reshape(-1,img_size_flat)\nc_test_y1 = np.array([i[1] for i in class_test1])\n\nclass_test2=s_t_xy[600:660]\nc_test_x2 = np.array([i[0] for i in class_test2])\nc_test_x2=c_test_x2.reshape(-1,img_size_flat)\nc_test_y2 = np.array([i[1] for i in class_test2])\n\nclass_test3=s_t_xy[800:830]\nc_test_x3 = np.array([i[0] for i in class_test3])\nc_test_x3=c_test_x3.reshape(-1,img_size_flat)\nc_test_y3 = np.array([i[1] for i in class_test3])\n\nclass_test4=s_t_xy[1020:1040]\nc_test_x4 = np.array([i[0] for i in class_test4])\nc_test_x4=c_test_x4.reshape(-1,img_size_flat)\nc_test_y4 = np.array([i[1] for i in class_test4])\n\nclass_test5=s_t_xy[1260:1300]\nc_test_x5 = np.array([i[0] for i in class_test5])\nc_test_x5=c_test_x5.re

In [ ]:
import numpy as np
import random
import numpy as np
import tensorflow as tf
import cv2
import os
import pandas as pd
from random import shuffle
from tqdm import tqdm

#t=np.loadtxt('/content/gdrive/My Drive/Colab Notebooks/flower/tttt.npy')

import matplotlib.pyplot as plt
# Convolutional Layer 1.
filter_size1 = 4 
num_filters1 = 32
# Convolutional Layer 2.
filter_size2 = 4
num_filters2 = 64
# Convolutional Layer 3.
filter_size3 = 4
num_filters3 = 128
# Convolutional Layer 4
filter_size4 = 4
num_filters4 = 256
# Convolutional Layer 5
filter_size5 = 4
num_filters5 = 128
# Fully-connected layer.
fc_size = 1024         
# Number of color channels for the images: 1 channel for gray-scale.  ########################우리는 흑백이랑 컬러 2개...?
num_channels = 1
# image dimensions (only squares for now)   #################
img_size = 128
# Size of image when flattened to a single dimension
img_size_flat = img_size * img_size * num_channels
# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size, img_size)
# class 써줌
classes = ['poppy',  'sunflower', 'lily','roseofsharon','tulip','rose','korearose','lotus']
num_classes = len(classes)
# batch size
batch_size = 50
keep_prob=tf.placeholder(tf.float32,name='keep_prob')


#data 불러옴
checkpoint_dir = "models/"

#train_data = np.load('flower_train_data.npy')

def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))
def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

def new_conv_layer(input,num_input_channels,filter_size,num_filters,use_pooling=True):  

    shape = [filter_size, filter_size, num_input_channels, num_filters]
    weights = new_weights(shape=shape)
    biases = new_biases(length=num_filters)

    layer = tf.nn.conv2d(input=input,filter=weights,strides=[1, 1, 1, 1],padding='SAME')
    layer += biases

    if use_pooling:
        layer = tf.nn.max_pool(value=layer,ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='SAME')

    layer = tf.nn.relu(layer)
    
    return layer, weights

def new_fc_layer(input,num_inputs,num_outputs,use_relu=True): 

    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    layer = tf.matmul(input, weights) + biases
    
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

def flatten_layer(layer):
    
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer_flat = tf.reshape(layer, [-1, num_features])

    return layer_flat, num_features

x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, axis=1)

layer_conv1, weights_conv1 = new_conv_layer(input=x_image,num_input_channels=num_channels,filter_size=filter_size1,
                                            num_filters=num_filters1,use_pooling=True)
layer_conv1=tf.nn.dropout(layer_conv1,keep_prob=keep_prob)
layer_conv2, weights_conv2 = new_conv_layer(input=layer_conv1,num_input_channels=num_filters1,filter_size=filter_size2,
                                            num_filters=num_filters2,use_pooling=True)
layer_conv2=tf.nn.dropout(layer_conv2,keep_prob=keep_prob)
layer_conv3, weights_conv3 = new_conv_layer(input=layer_conv2,num_input_channels=num_filters2,filter_size=filter_size3,
                                            num_filters=num_filters3,use_pooling=True)
layer_conv3=tf.nn.dropout(layer_conv3,keep_prob=keep_prob)
layer_conv4, weights_conv4 = new_conv_layer(input=layer_conv3,num_input_channels=num_filters3,filter_size=filter_size4,
                                            num_filters=num_filters4,use_pooling=True)
layer_conv5=tf.nn.dropout(layer_conv4,keep_prob=keep_prob)
layer_conv5, weights_conv5 = new_conv_layer(input=layer_conv4,num_input_channels=num_filters4,filter_size=filter_size5,
                                            num_filters=num_filters5,use_pooling=True)
layer_conv5=tf.nn.dropout(layer_conv5,keep_prob=keep_prob)
layer_flat, num_features = flatten_layer(layer_conv5)
layer_fc1 = new_fc_layer(input=layer_flat,num_inputs=num_features,num_outputs=fc_size,use_relu=True)
layer_fc2 = new_fc_layer(input=layer_fc1,num_inputs=fc_size,num_outputs=num_classes,use_relu=False)

y_pred = tf.nn.softmax(layer_fc2,name='operation')
y_pred_cls = tf.argmax(y_pred, axis=1)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,labels=y_true)
cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



shuffle(n_xy)
train=n_xy

test=n_t_xy

shuffle(test)

print(train.shape,test.shape)

x_batch = np.array([i[0] for i in train])
x_batch=x_batch.reshape(-1,img_size_flat)
#x_batch = np.array([i[0] for i in train])

y_true_batch = np.array([i[1] for i in train])

test_x = np.array([i[0] for i in test])
test_x=test_x.reshape(-1,img_size_flat)
#test_x = np.array([i[0] for i in test])

test_y = np.array([i[1] for i in test])


print(len(test_x),len(test_y))
#for i in range(100):
#  plt.imshow((c_test_x9[i]).reshape(28,28))
#  plt.show()
with tf.Session() as session:    
    global total_iterations
    init_op = tf.initialize_all_variables()
    session.run(init_op)
    session.run(tf.global_variables_initializer())
    total_iterations = 0
    num_iterations=200
    for i in range(total_iterations,total_iterations + num_iterations):
      
      a = 0
      for __ in range(int(len(train)/batch_size)):
        
        
          feed_dict_train = {x: x_batch[a:a+batch_size,:],y_true: y_true_batch[a:a+batch_size],keep_prob:1.0}
          session.run(optimizer, feed_dict=feed_dict_train)
          a = a + batch_size
      #plt.imshow(x_batch[i : i + 1].reshape(28, 28))
      #plt.show()
      if i % 2 == 0:
         print("Iteration = ", i, "Loss = ", session.run(cost, feed_dict=feed_dict_train),
                      "Train Accuracy = ", session.run(accuracy, feed_dict=feed_dict_train), 
                      "Test Accuracy = ", session.run(accuracy, feed_dict={x:test_x, y_true: test_y,keep_prob:1.0}))


   
       

#optimize(num_iterations=100)

W0708 10:53:09.415184 140529386407808 deprecation.py:506] From <ipython-input-2-e7c84b0f4f72>:100: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0708 10:53:09.590442 140529386407808 deprecation.py:323] From <ipython-input-2-e7c84b0f4f72>:119: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



(980, 2) (980, 2)
980 

W0708 10:53:11.351383 140529386407808 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/util/tf_should_use.py:193: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


980
Iteration =  0 Loss =  25.501072 Train Accuracy =  0.14 Test Accuracy =  0.06530612
Iteration =  2 Loss =  6.023158 Train Accuracy =  0.38 Test Accuracy =  0.27857143
Iteration =  4 Loss =  2.4063172 Train Accuracy =  0.56 Test Accuracy =  0.6010204
Iteration =  6 Loss =  0.8141873 Train Accuracy =  0.82 Test Accuracy =  0.6704082
Iteration =  8 Loss =  0.5366695 Train Accuracy =  0.9 Test Accuracy =  0.83265305
Iteration =  10 Loss =  0.34566867 Train Accuracy =  0.92 Test Accuracy =  0.85918367
Iteration =  12 Loss =  0.16597402 Train Accuracy =  0.96 Test Accuracy =  0.8642857
Iteration =  14 Loss =  0.09603994 Train Accuracy =  0.98 Test Accuracy =  0.8683674
Iteration =  16 Loss =  0.055714283 Train Accuracy =  1.0 Test Accuracy =  0.8704082
Iteration =  18 Loss =  0.035027843 Train Accuracy =  1.0 Test Accuracy =  0.86938775
Iteration =  20 Loss =  0.02551163 Train Accuracy =  1.0 Test Accuracy =  0.87142855
Iteration =  22 Loss =  0.020048799 Train Accuracy =  1.0 Test Accur